Mouting the google drive.

In [27]:
# mounting the drive  
from google.colab import drive
drive.mount('/content/drive')  

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing all the required packages 

In [28]:
import pandas as pd   # for reading the dataset
from sklearn.model_selection import train_test_split # for spliting the dataset into testing and training phases
import numpy as np   # for datamanuplation operations.
import nltk 
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
from keras.utils import to_categorical
import nltk
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Importing the dataset by using pandas and storing it in the data_set variable

In [0]:
data_set = pd.read_csv('https://raw.githubusercontent.com/cacoderquan/Sentiment-Analysis-on-the-Rotten-Tomatoes-movie-review-dataset/master/train.tsv', sep='\t')

Displaying both the initial data and data after once the preprocessing is done.

In [30]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re 

pd.set_option('max_colwidth',400)

wordnet = WordNetLemmatizer()
stopwords_en = stopwords.words("english")
punctuations = "?:!.,;-()"

raw_reviews = data_set.Phrase.values
cleaned_reviews = []

for i in range(len(raw_reviews)):
  review = str(raw_reviews[i])
  review=re.sub('[^a-zA-Z]',' ',review)
  review=[wordnet.lemmatize(w) for w in word_tokenize(str(review).lower())]
  review=' '.join(review)
  cleaned_reviews.append(review)

data_set['cleaned_reviews'] = cleaned_reviews

data_set.head()

,PhraseId,SentenceId,Phrase,Sentiment,cleaned_reviews
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1,a series of escapade demonstrating the adage that what is good for the goose is also good for the gander some of which occasionally amuses but none of which amount to much of a story
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2,a series of escapade demonstrating the adage that what is good for the goose
2,3,1,A series,2,a series
3,4,1,A,2,a
4,5,1,series,2,series


In [0]:
X = data_set.cleaned_reviews.values
Y = to_categorical(data_set.Sentiment.values)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = TfidfVectorizer(max_features=4000,stop_words = None, ngram_range=(1,2))
X = vectorizer.fit_transform(X)

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=2003)

In [0]:
import numpy as np
x_train_np = x_train.toarray()
y_train_np = np.array(y_train)
x_test_np = x_test.toarray()
y_test_np = np.array(y_test)

In [0]:
x_train = np.expand_dims(x_train_np, axis=2)
x_test = np.expand_dims(x_test_np, axis=2)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout

model = Sequential()

def cnn(data_x):
  #Convolution layer 1
  model.add(Conv1D(filters = 64, kernel_size=3, activation='relu', input_shape=(data_x.shape[1],1)))
  #MaxPool layer
  model.add(MaxPooling1D(pool_size =2))
  #Convolution layer 2
  model.add(Conv1D(filters = 128, kernel_size=3, activation='relu'))
  #MaxPool layer
  model.add(MaxPooling1D(pool_size =2))
  model.add(Flatten())
  #Fully connected layer
  model.add(Dense(100, activation='relu'))
  #Dropout layer
  model.add(Dropout(0.2))
  #Output Layer
  model.add(Dense(5, activation='softmax')) 

In [0]:
from keras import metrics
from sklearn.metrics import f1_score

In [0]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
cnn(x_train_np)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[metrics.categorical_accuracy,f1_m,precision_m, recall_m])

In [46]:
history = model.fit(x_train, y_train_np, epochs=5, batch_size=128)

Epoch 1/50
109242/109242 [==============================] - 27s 251us/step - loss: 0.4105 - categorical_accuracy: 0.8253 - f1_m: 0.8244 - precision_m: 0.8428 - recall_m: 0.8071
Epoch 2/50
109242/109242 [==============================] - 27s 250us/step - loss: 0.4087 - categorical_accuracy: 0.8260 - f1_m: 0.8247 - precision_m: 0.8428 - recall_m: 0.8075
Epoch 3/50
109242/109242 [==============================] - 27s 250us/step - loss: 0.4074 - categorical_accuracy: 0.8272 - f1_m: 0.8257 - precision_m: 0.8436 - recall_m: 0.8087
Epoch 4/50
109242/109242 [==============================] - 27s 249us/step - loss: 0.4042 - categorical_accuracy: 0.8270 - f1_m: 0.8260 - precision_m: 0.8437 - recall_m: 0.8092
Epoch 5/50
109242/109242 [==============================] - 27s 248us/step - loss: 0.4039 - categorical_accuracy: 0.8276 - f1_m: 0.8269 - precision_m: 0.8446 - recall_m: 0.8100
Epoch 6/50
109242/109242 [==============================] - 27s 248us/step - loss: 0.4031 - categorical_accuracy: 0

In [47]:
loss, accuracy, f1_score, precision, recall = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy " + str(accuracy) + ":\n\ff1_score = " + str(f1_score) +
          "\nPrecision = " + str(precision) + "\nRecall = " + str(recall))

Accuracy 0.6195693963860054:
f1_score = 0.6169272960390864
Precision = 0.623162252807552
Recall = 0.6110043145798625


In [49]:
model.save("1104076_1d_conv_reg")
print("Saved model to disk")

Saved model to disk
